In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [5]:
finviz_url = 'https://finviz.com/quote.ashx?t='
news_tables = {}
headers = {'user-agent': 'my-app'}
    
tickers = ['AMZN']

for ticker in tickers:
    try:
        url = finviz_url + ticker
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raises an HTTPError if the HTTP request returned an unsuccessful status code
        html = BeautifulSoup(response.content, 'html.parser')
        news_table = html.find(id='news-table')
        news_tables[ticker] = news_table
    except Exception as e:
        print(f"Could not fetch data for {ticker}: {e}")

print(news_tables)

{'AMZN': <table border="0" cellpadding="1" cellspacing="0" class="fullview-news-outer news-table" id="news-table" width="100%">
<tr class="cursor-pointer has-label" onclick="trackAndOpenNews(event, 'CNBC TV', 'https://www.youtube.com/watch?v=-vsKhzYrQJY');">
<td align="right" width="130">
            Today 08:55PM
        </td>
<td align="left">
<div class="news-link-container">
<div class="news-link-left">
<a class="tab-link-news" href="https://www.youtube.com/watch?v=-vsKhzYrQJY" rel="nofollow" target="_blank">Lightning Round: Amazon has a lot of room to go higher, says Jim Cramer</a>
</div>
<div class="news-link-right">
<span>(CNBC TV)</span></div></div></td></tr>
<tr class="cursor-pointer has-label" onclick="trackAndOpenNews(event, 'Chain Store Age', 'https://chainstoreage.com/amazon-ai-initiative-tackles-defective-products');">
<td align="right" width="130">
            08:08PM
        </td>
<td align="left">
<div class="news-link-container">
<div class="news-link-left">
<a class=

In [13]:
import re
import csv
from time import sleep
from bs4 import BeautifulSoup
import requests

headers = {
    'accept': '*/*',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.9',
    'referer': 'https://www.google.com',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36'
}

def get_article(card):
    """Extract article information from the raw html"""
    headline = card.find('h4', 's-title').text
    source = card.find("span", 's-source').text
    posted = card.find('span', 's-time').text.replace('·', '').strip()
    description = card.find('p', 's-desc').text.strip()
    raw_link = card.find('a').get('href')
    unquoted_link = requests.utils.unquote(raw_link)
    pattern = re.compile(r'RU=(.+)\/RK')
    match = re.search(pattern, unquoted_link)
    
    if match:
        clean_link = match.group(1)
    else:
        clean_link = None  # or set to some default value or handle it as needed
    
    article = (headline, source, posted, description, clean_link)
    return article


def get_the_news(search):
    """Run the main program"""
    template = 'https://news.search.yahoo.com/search?p={}'
    url = template.format(search)
    articles = []
    links = set()
    
    while True:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        cards = soup.find_all('div', 'NewsArticle')
        
        # extract articles from page
        for card in cards:
            article = get_article(card)
            link = article[-1]
            if not link in links:
                links.add(link)
                articles.append(article)        
                
        # find the next page
        try:
            url = soup.find('a', 'next').get('href')
            sleep(1)
        except AttributeError:
            break
            
    # save article data
    with open('results.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Headline', 'Source', 'Posted', 'Description', 'Link'])
        writer.writerows(articles)
        
    return articles

In [14]:
# run the main program
articles = get_the_news('iphone')

In [16]:
len(articles)

963

In [15]:
articles[0:4]

[('iPhone 16 Pro will break a world record with this new feature, according to new reports',
  'Mashable',
  '10 hours ago',
  'The iPhone 16 Pro and iPhone 16 Pro Max are expected to break a world record, according to two...',
  'https://mashable.com/article/iphone-16-pro-thinnest-bezels'),
 ('iPhone 7 Settlement: Your Deadline to File a Class-Action Lawsuit Claim Is June 3',
  'InvestorPlace',
  '4 days ago',
  'Owners of the iPhone 7 only have a few days left to file a claim for a settlement in the...',
  'https://investorplace.com/2024/05/iphone-7-settlement-your-deadline-to-file-a-class-action-lawsuit-claim-is-june-3rd/'),
 ('iPhone 16: Get the Inside Scoop on the Most Credible Apple Rumors',
  'CNET',
  '4 days ago',
  "It's been more than eight months since the iPhone 15 launched, and as Apple's Worldwide Developers...",
  'https://www.cnet.com/tech/mobile/iphone-16-get-the-inside-scoop-on-the-most-credible-apple-rumors/'),
 ("The Rumored iPhone 16 Feature I'm Most Excited For",

In [24]:
for a in articles[:2]:
    print(a[4])

https://mashable.com/article/iphone-16-pro-thinnest-bezels
https://investorplace.com/2024/05/iphone-7-settlement-your-deadline-to-file-a-class-action-lawsuit-claim-is-june-3rd/


In [29]:
import requests
from bs4 import BeautifulSoup

def crawl_content(url):
    try:
        # Send a GET request to the URL
        response = requests.get(url)
        response.raise_for_status()  # Raise an HTTPError for bad responses

        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract the main content of the article
        # The specific tags and classes used here may need to be adjusted based on the site's HTML structure
        article_body = soup.find('div', class_='article-body')
        
        if not article_body:
            article_body = soup.find('article')  # Fallback to a more general tag

        paragraphs = article_body.find_all('p')
        content = '\n'.join([para.get_text() for para in paragraphs])
        
        return content
    except Exception as e:
        return f"Error occurred: {e}"

# URLs to crawl
urls = articles

# Crawl and print content from each URL
for url in urls[:5]:
    print(f"Crawling URL: {url[4]}")
    content = crawl_content(url[4])
    print(f"Content from {url[4]}:\n{content}\n")
    print('------------------------------------------------------------\n')


Crawling URL: https://mashable.com/article/iphone-16-pro-thinnest-bezels
Content from https://mashable.com/article/iphone-16-pro-thinnest-bezels:
The iPhone 16 Pro and iPhone 16 Pro Max are expected to break a world record, according to two leakers.
One tipster from Weibo, a China-based social media website, claims that they have measured the bezels of both iPhone 16 Pro models — and they're set to be the "narrowest in the world."
Ice Universe, a well-known bean spiller in the tech space, concurs with this leak. "My friend confirmed that the iPhone 16 Pro will reduce bezel, surpassing Galaxy S24 to become the world's narrowest bezel mobile phone, which seems to be very close to the dream form."
The Weibo tipster pointed out that the iPhone 15 Pro models' bezels are about 1.71mm thick, give or take. 
For the next-generation iPhone series, they are reporting the following measurements:
iPhone 16 Pro - 1.2mm
iPhone 16 Pro Max - 1.15mm
However, I'd suggest you have your grain of salt ready